# Get the XML Docs

In [2]:
import graphlab
from graphlab import SFrame

In [3]:
import xml.dom.minidom as minidom
request_xml = minidom.parse('xml/ImageRequest.xml')
info_xml = minidom.parse('xml/ImageInfo.xml')

In [4]:
def upload_image(image_filename, request_xml):
    import base64
    import requests
    import xml.dom.minidom as minidom
    
    # encode the image as base64
    image_file = open(image_filename, 'rb')
    encoded_string = base64.b64encode(image_file.read())
    
    #insert the encoded string into the xml
    request_xml.getElementsByTagName('imagefile_data')[0].firstChild.nodeValue = encoded_string
    
    #headers for the request
    headers = {
        'Content-Type': 'application/xml'
    }
    
    #send the request
    r = requests.post('http://www.betafaceapi.com/service.svc/UploadNewImage_File', data=request_xml.toxml(), headers=headers)
    
    #get the response xml as DOM
    response_xml = minidom.parseString(r.text)
    uid = response_xml.getElementsByTagName('img_uid')[0].firstChild.nodeValue
    print "Image Filename: "+image_filename+"\t UID: "+str(uid)
    
    return uid  

def get_image_info(image_uid, info_xml):
    import xml.dom.minidom as minidom
    import requests
    
    # insert the UID into the xml
    info_xml.getElementsByTagName('img_uid')[0].firstChild.nodeValue = image_uid
    
    #prepare headers
    headers = {
        'Content-Type': 'application/xml'
    }
    
    #send request
    r = requests.post('http://www.betafaceapi.com/service.svc/GetImageInfo', data=info_xml.toxml(), headers=headers)
    
    return minidom.parseString(r.text)

def get_feature_points(xml):
    import numpy as np
    
    pointInfos = xml.getElementsByTagName('PointInfo')
    
    feature_points = {}
    
    for info in pointInfos:
        name = info.getElementsByTagName('name')[0].firstChild.nodeValue
        name = name.replace(' ', '_')
        x = info.getElementsByTagName('x')[0].firstChild.nodeValue;
        y = info.getElementsByTagName('y')[0].firstChild.nodeValue;
        
        feature_points[str(name)] = {
            'x': float(x),
            'y': float(y)
        }
    return feature_points

def calculate_distance(feature_1, feature_2, feature_points):
    import numpy as np
    x = np.array([feature_points[feature_1]['x'],feature_points[feature_1]['y']])
    y = np.array([feature_points[feature_2]['x'],feature_points[feature_2]['y']])
    return np.linalg.norm(x-y)

# Get Image UID

In [9]:
uid = upload_image(image_filename='images/0007.jpg', request_xml=request_xml)

Image Filename: images/0007.jpg	 UID: 8bc41a26-45a8-4444-9bb2-a123046a4200


# Get feature points

In [10]:
feature_points = get_feature_points(get_image_info(uid, info_xml=info_xml))
len(feature_points.keys())

122

# Calculate necessary features and build SFrame

In [11]:
features = {
        'eye_outer': [calculate_distance('basic_eye_left_outer', 'basic_eye_right_outer', feature_points)],
        'eye_inner': [calculate_distance('basic_eye_left_inner', 'basic_eye_right_inner', feature_points)],
        'eye': [calculate_distance('basic_eye_left', 'basic_eye_right', feature_points)],
        'mouth': [calculate_distance('basic_mouth_left', 'basic_mouth_right', feature_points)],
        'nose_mouth': [calculate_distance('basic_nose_tip', 'basic_mouth_center', feature_points)],
        'eyebrow_inner': [calculate_distance('basic_eyebrow_left_inner_corner', 'basic_eyebrow_right_inner_corner', feature_points)],
        'eyebrow_outer': [calculate_distance('basic_eyebrow_left_outer_corner', 'basic_eyebrow_right_outer_corner', feature_points)],
        'chin': [calculate_distance('basic_chin_left', 'basic_chin_right', feature_points)],
        'nose': [calculate_distance('basic_nose_left', 'basic_nose_right', feature_points)],
        'chin_mouth': [calculate_distance('basic_chin_bottom', 'basic_mouth_center', feature_points)]
    }
data = SFrame(features)

In [12]:
ethnicity_model = graphlab.load_model('model/ethnicity_model/')
ethnicity_model.get("coefficients").print_rows(num_rows=30)

+---------------+-------+-------+------------------+-----------------+
|      name     | index | class |      value       |      stderr     |
+---------------+-------+-------+------------------+-----------------+
|  (intercept)  |  None |   1   |  -21.3759477012  |  7.34592570414  |
|      chin     |  None |   1   |  0.130545894738  | 0.0491052418295 |
|   chin_mouth  |  None |   1   | -0.0936429572762 | 0.0741099931769 |
|      eye      |  None |   1   | -0.0175815041063 |  0.352233322903 |
|   eye_inner   |  None |   1   |  0.711311591192  |  0.22891455113  |
|   eye_outer   |  None |   1   | -0.0476925114056 |  0.247895652295 |
| eyebrow_inner |  None |   1   | -0.104485073087  |  0.136263480391 |
| eyebrow_outer |  None |   1   | -0.0828540505092 | 0.0741399281388 |
|     mouth     |  None |   1   | -0.291854962374  | 0.0830294828059 |
|      nose     |  None |   1   |  0.371173990239  |  0.15459639892  |
|   nose_mouth  |  None |   1   |  0.127011547386  | 0.0736648543777 |
+-----

# Are you Chinese or not

In [8]:
confidence = ethnicity_model.predict(data, output_type='probability')
ethnicity = ethnicity_model.predict(data)
if ethnicity[0] == 1:
    print "Ethnicity: Chinese \t Confidence: "+str(confidence[0])
else:
    print "Ethnicity: Non-Chinese \t Confidence: "+str(1-confidence[0])

Ethnicity: Non-Chinese 	 Confidence: 0.979491347285
